In [1955]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)

import xgboost
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer

from prepare import x_y_split, rmse, select_kbest, rfe

In [1956]:
df = pd.read_csv('season.csv',index_col=0)

seasons = range(2016,2023)
positions = ['QB','RB','WR','TE']
add = []

df = df[df['g'] > 9]

for s in seasons:
    for p in positions:
        test = df[(df.year==s)&(df.pos==p)].sort_values('avg_draft_pos_ppr').reset_index(drop=True)
        test['adp_by_pos'] = test.index+1
        
        test = test[(test.year==s)&(test.pos==p)].sort_values('ppr_pts',ascending=False).reset_index(drop=True)
        test['pos_rank'] = test.index+1
        
        add.append(test)

df = pd.concat(add).reset_index(drop=True)

for i in range(0,len(df.index)):
    if df.loc[i,'pos'] =='QB' or df.loc[i,'pos'] =='TE':
        
        if df.loc[i, 'adp_by_pos'] <= 3:
            if df.loc[i,'pos_rank'] <= 3:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        else:
            if df.loc[i,'pos_rank']<=12:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

    else:
        if df.loc[i, 'adp_by_pos'] <= 6:
            if df.loc[i,'pos_rank'] <= 6:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        elif df.loc[i, 'adp_by_pos'] > 36:
            if df.loc[i,'pos_rank'] < 36:
                df.loc[i,'success'] = 1
            else:
                df.loc[i, 'success'] = 0

        else:
            if df.loc[i,'pos_rank'] <= df.loc[i,'adp_by_pos']:
                df.loc[i, 'success'] = 1
            else:
                df.loc[i, 'success'] = 0
        
        
for i in range(0,len(df.index)):
    if df.loc[i,'avg_draft_pos_ppr'] <= 12:
        df.loc[i, 'round'] = 1
    elif df.loc[i,'avg_draft_pos_ppr'] <= 24:
        df.loc[i, 'round'] = 2
    elif df.loc[i,'avg_draft_pos_ppr'] <= 36:
        df.loc[i, 'round'] = 3
    elif df.loc[i,'avg_draft_pos_ppr'] <= 48:
        df.loc[i, 'round'] = 4
    elif df.loc[i,'avg_draft_pos_ppr'] <= 60:
        df.loc[i, 'round'] = 5
    elif df.loc[i,'avg_draft_pos_ppr'] <= 72:
        df.loc[i, 'round'] = 6
    elif df.loc[i,'avg_draft_pos_ppr'] <= 84:
        df.loc[i, 'round'] = 7
    elif df.loc[i,'avg_draft_pos_ppr'] <= 96:
        df.loc[i, 'round'] = 8
    elif df.loc[i,'avg_draft_pos_ppr'] <= 108:
        df.loc[i, 'round'] = 9
    elif df.loc[i,'avg_draft_pos_ppr'] <= 120:
        df.loc[i, 'round'] = 10
    elif df.loc[i,'avg_draft_pos_ppr'] <= 132:
        df.loc[i, 'round'] = 11
    elif df.loc[i,'avg_draft_pos_ppr'] <= 144:
        df.loc[i, 'round'] = 12
    elif df.loc[i,'avg_draft_pos_ppr'] <= 156:
        df.loc[i, 'round'] = 13
    elif df.loc[i,'avg_draft_pos_ppr'] <= 168:
        df.loc[i, 'round'] = 14
    else:
        df.loc[i, 'round'] = 15


In [1957]:
qb_df = df[df['pos']=='QB']
rb_df = df[df['pos']=='RB']
wr_df = df[df['pos']=='WR']
te_df = df[df['pos']=='TE']

In [1958]:
qb_df.drop(columns=['rk','pos','tgt','rec','rec_yards','y/r','rec_tds','standard_pts','vbd', 'team'], inplace=True)

In [1959]:
qb_df['rating'] = round(((((((qb_df['cmp']/qb_df['pass_att'])-.3)*5) + 
                  ((qb_df['pass_yds']/qb_df['pass_att']-3)*.25) +
                  ((qb_df['pass_tds']/qb_df['pass_att'])*20) +
                  (2.375-((qb_df['int']/qb_df['pass_att'])*25)))/6)*100),2)

In [1960]:
def add_target(group):
    group['target'] = group['ppr_pts'].shift(-1)
    group = group.fillna(0)
    return group

In [1961]:
qb_df = add_target(qb_df)


In [1962]:
qb_df = qb_df[(qb_df['player']!= 'Tom Brady') & (qb_df['player']!='Marcus Mariota')]

In [1963]:
qb_df['comp%'] = round((qb_df['cmp'] / qb_df['pass_att']) * 100, 2)
qb_df['int%'] = round((qb_df['int'] / qb_df['pass_att']) * 100, 2)

In [1964]:
qb_df.drop(columns=['cmp','pass_att', 'int'], inplace=True)

In [1965]:
qb_cols = ['age','g','gs','pass_yds','pass_tds','rush_att','rush_yard','y/a','rush_tds',
           'fmb','fl','rush_rec_tds','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round','ppr_pts',
           'comp%', 'int%', 'rating']

In [1966]:
def xgb_modeling(df, cols):
    
    X_train = df[df['year']<2021]
    X_val = df[df['year']<2022]
    X_test = df[df['year']==2022]
    
    y_train = X_train['target']
    X_train.drop(columns=['target'], inplace = True)
    
    y_val = X_val['target']
    X_val.drop(columns=['target'], inplace = True)

    X_test.drop(columns=['target'], inplace = True)
    
    ss = StandardScaler()
    
    X_train[cols] = ss.fit_transform(X_train[cols])
    X_val[cols] = ss.transform(X_val[cols])
    X_test[cols] = ss.transform(X_test[cols])
    
    cols.append('success')
    
    xgb = xgboost.XGBRegressor(eval_metric='rmse',n_estimators=100, max_depth=4, 
                               subsample=.65, colsample_bytree=.8, seed=42,
                               eta=.15, gamma=100)
    
    xgb.fit(X_train[cols], y_train)
    xgb_preds = xgb.predict(X_train[cols])
    
    preds = pd.DataFrame({'actual':y_train,
                          'baseline':y_train.mean(),
                          'xgb_preds':xgb_preds})
    
    xgb_val_preds = xgb.predict(X_val[cols])
    
    val_preds = pd.DataFrame({'actual':y_val,
                              'baseline':y_train.mean(),
                              'xgb_val_preds':xgb_val_preds})
    
    
    pos_2023 = pd.DataFrame({'player':X_test['player'],
                             'preds':xgb.predict(X_test[cols])})
    
    val_2022 = X_val[X_val['year']==2021]
    pos_2022 = pd.DataFrame({'player':val_2022['player'],
                             'actual':val_2022['ppr_pts'],
                             'preds':xgb.predict(val_2022[cols])})
    
    return preds, val_preds, pos_2023, pos_2022

In [1967]:
qb_preds, qb_val_preds, qb_2023, qb_2022 = xgb_modeling(qb_df, qb_cols)

In [1968]:
qb_2023.sort_values('preds',ascending=False).head(20)

,player,preds
2119,Jalen Hurts,363.743866
2117,Patrick Mahomes,355.721680
2118,Josh Allen,349.049744
2120,Joe Burrow,337.584564
2121,Geno Smith,288.472595
2125,Daniel Jones,282.682495
2126,Jared Goff,279.885620
2123,Trevor Lawrence,278.389069
2122,Justin Fields,278.300842
2124,Kirk Cousins,277.009369


In [1969]:
rmse(qb_preds,'baseline')

82.25269165664197

In [1970]:
rmse(qb_preds, 'xgb_preds')

3.6701439584018702

In [1971]:
rmse(qb_val_preds, 'xgb_val_preds')

8.344113457745234

In [1972]:
rb_df.head()

,player,rk,team,pos,age,g,gs,cmp,pass_att,pass_yds,pass_tds,int,rush_att,rush_yard,y/a,rush_tds,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,standard_pts,ppr_pts,vbd,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round
31,David Johnson,1,ARI,RB,25,16,16,0.0,0.0,0.0,0.0,0.0,293.0,1239.0,4.23,16.0,120.0,80.0,879.0,10.99,4.0,5.0,3.0,20,328.0,407.8,191.0,1,2016,5.4,3.4,1,1.0,1.0
32,Ezekiel Elliott,2,DAL,RB,21,15,15,0.0,0.0,0.0,0.0,0.0,322.0,1631.0,5.07,15.0,39.0,32.0,363.0,11.34,1.0,5.0,1.0,16,293.0,325.4,156.0,2,2016,6.3,8.3,3,1.0,1.0
33,Le'Veon Bell,5,PIT,RB,24,12,12,0.0,1.0,0.0,0.0,0.0,261.0,1268.0,4.86,7.0,94.0,75.0,616.0,8.21,2.0,4.0,1.0,9,242.0,317.4,105.0,3,2016,10.1,12.3,5,1.0,2.0
34,LeSean McCoy,4,BUF,RB,28,15,15,0.0,0.0,0.0,0.0,0.0,234.0,1267.0,5.41,13.0,57.0,50.0,356.0,7.12,1.0,3.0,0.0,14,248.0,298.3,111.0,4,2016,19.7,22.0,7,1.0,2.0
35,DeMarco Murray,6,TEN,RB,28,16,16,1.0,2.0,10.0,1.0,0.0,293.0,1287.0,4.39,9.0,67.0,53.0,377.0,7.11,3.0,3.0,1.0,12,241.0,293.8,104.0,5,2016,36.5,40.3,10,1.0,4.0


In [1973]:
rb_df.drop(columns=['rk','team','pos','cmp','pass_att','pass_yds','pass_tds','int','standard_pts','vbd'], inplace=True)

In [1974]:
rb_df = add_target(rb_df)

In [1975]:
rb_cols = ['age','g','gs','rush_att','rush_yard','y/a','rush_tds','tgt','rec','rec_yards','y/r','rec_tds','fmb',
           'fl','rush_rec_tds','ppr_pts','pos_rank','avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round', 'rec%']

In [1976]:
rb_df['rec%'] = round(((rb_df['rec']/rb_df['tgt'])*100),2)

In [1977]:
rb_preds, rb_val_preds, rb_2023, rb_2022 = xgb_modeling(rb_df, rb_cols)

In [1978]:
rmse(rb_preds,'baseline')

88.70617248916972

In [1979]:
rmse(rb_preds,'xgb_preds')

3.3940635057115687

In [1980]:
rmse(rb_val_preds,'xgb_val_preds')

18.413206444002018

In [1981]:
rb_2023.sort_values('preds',ascending=False).head(20)

,player,preds
2146,Austin Ekeler,358.770905
2147,Christian McCaffrey,336.205872
2148,Josh Jacobs,292.141418
2149,Derrick Henry,275.291931
2151,Nick Chubb,265.352509
2150,Saquon Barkley,260.854431
2154,Aaron Jones,240.025253
2153,Tony Pollard,239.500168
2152,Rhamondre Stevenson,239.297623
2155,Joe Mixon,233.122986


In [1982]:
wr_df.drop(columns=['rk','team','pos','cmp','pass_tds','pass_att','pass_yds','int','rush_att','rush_yard','y/a','rush_tds',
                    'standard_pts','vbd'],inplace=True)

In [1983]:
wr_df = add_target(wr_df)

In [1984]:
wr_cols = ['age','g','gs','tgt','rec','rec_yards','y/r','rec_tds','fmb','fl','rush_rec_tds','ppr_pts','pos_rank',
           'avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round', 'rec%']

In [1985]:
wr_df['rec%'] = round(((wr_df['rec']/wr_df['tgt'])*100),2)

In [1986]:
wr_df = wr_df[wr_df['player']!='KaVontae Turpin']

In [1987]:
wr_preds, wr_val_preds, wr_2023, wr_2022 = xgb_modeling(wr_df, wr_cols)

In [1988]:
rmse(wr_preds, 'baseline')

82.77064023071632

In [1989]:
rmse(wr_preds,'xgb_preds')

2.583932184527564

In [1990]:
rmse(wr_val_preds,'xgb_val_preds')

4.11715519936657

In [1991]:
wr_2023.sort_values('preds',ascending=False).head(20)

,player,preds
2251,Tyreek Hill,319.376892
2253,Stefon Diggs,314.559387
2252,Davante Adams,303.151611
2254,CeeDee Lamb,302.002350
2255,A.J. Brown,292.385651
2250,Justin Jefferson,284.103638
2256,Amon-Ra St. Brown,263.584991
2257,Jaylen Waddle,250.549271
2258,DeVonta Smith,243.873322
2259,Amari Cooper,243.163849


In [1992]:
te_df.drop(columns=['rk','team','pos','cmp','pass_att','pass_yds','pass_tds','int','rush_att','rush_yard','y/a',
                    'rush_tds','standard_pts','vbd'], inplace=True)

In [1993]:
te_df = add_target(te_df)

In [1994]:
te_df

,player,age,g,gs,tgt,rec,rec_yards,y/r,rec_tds,fmb,fl,rush_rec_tds,ppr_pts,pos_rank,year,avg_draft_pos,avg_draft_pos_ppr,adp_by_pos,success,round,target
262,Travis Kelce,27,16,15,117.0,85.0,1125.0,13.24,4.0,0.0,0.0,4,223.0,1,2016,61.4,64.3,4,1.0,6.0,209.0
263,Kyle Rudolph,27,16,16,132.0,83.0,840.0,10.12,7.0,0.0,0.0,7,209.0,2,2016,339.0,322.0,21,1.0,15.0,207.3
264,Greg Olsen,31,16,16,129.0,80.0,1073.0,13.41,3.0,0.0,0.0,3,207.3,3,2016,51.3,48.4,2,1.0,5.0,189.3
265,Jimmy Graham,30,16,15,95.0,65.0,923.0,14.20,6.0,2.0,2.0,6,189.3,4,2016,124.0,144.3,12,1.0,13.0,188.1
266,Delanie Walker,32,15,9,102.0,65.0,800.0,12.31,7.0,1.0,0.0,7,188.1,5,2016,68.9,64.0,3,0.0,6.0,183.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478,Stephen Anderson,29,16,0,7.0,3.0,9.0,3.00,0.0,0.0,0.0,0,3.9,91,2022,350.0,350.0,44,0.0,15.0,3.1
2479,Sean McKeon,25,13,0,3.0,2.0,11.0,5.50,0.0,0.0,0.0,0,3.1,92,2022,350.0,350.0,43,0.0,15.0,2.5
2480,Devin Asiasi,25,12,0,4.0,2.0,5.0,2.50,0.0,0.0,0.0,0,2.5,93,2022,350.0,350.0,42,0.0,15.0,1.4
2481,Richard Rodgers,30,10,1,4.0,1.0,4.0,4.00,0.0,0.0,0.0,0,1.4,94,2022,350.0,350.0,87,0.0,15.0,0.0


In [1995]:
te_df['rec%'] = round(((te_df['rec']/te_df['tgt'])*100),2)

In [1996]:
te_cols = ['age','g','gs','tgt','rec','rec_yards','y/r','rec_tds','fmb','fl','rush_rec_tds','ppr_pts','pos_rank',
           'avg_draft_pos','avg_draft_pos_ppr','adp_by_pos','round', 'rec%']

In [1997]:
te_df = te_df[(te_df['player']!='Richard Rodgers') & (te_df['player']!='Feleipe Franks')]

In [1998]:
te_preds, te_val_preds, te_2023, te_2022 = xgb_modeling(te_df, te_cols)

In [1999]:
rmse(te_preds, 'baseline')

61.83313016511478

In [2000]:
rmse(te_preds, 'xgb_preds')

2.808151062053373

In [2001]:
rmse(te_val_preds,'xgb_val_preds')

3.65309473495996

In [2002]:
te_2023.sort_values('preds',ascending=False).head(30)

,player,preds
2388,Travis Kelce,266.041504
2389,T.J. Hockenson,201.923584
2391,Mark Andrews,187.974609
2390,George Kittle,187.570221
2392,Evan Engram,170.315125
2393,Tyler Higbee,144.985458
2394,Pat Freiermuth,142.847565
2395,Cole Kmet,140.763672
2397,Dalton Schultz,137.745346
2396,Taysom Hill,137.402206
